<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/250811_%EB%A6%AC%EC%8A%A4%ED%81%AC_%EC%8B%A0%ED%98%B8%EB%93%B1_%EC%A7%80%ED%91%9C_finBERT_dartAPI_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# RiskScanner Lite: 뉴스 기반 간이 리스크 스코어러 (짧고 안전하게)

import os, requests, io, json
from typing import List, Dict, Any, Tuple, Optional
from datetime import datetime, timedelta, timezone
from xml.etree import ElementTree as ET


# ===== 설정 (환경변수 없으면 자동 폴백) =====
NAVER_ID = os.getenv("NAVER_CLIENT_ID", "")
NAVER_SECRET = os.getenv("NAVER_CLIENT_SECRET", "")
USER_AGENT = "RiskScannerLite/1.0"
HTTP_TIMEOUT = 10
DEFAULT_DAYS = 30
DEFAULT_MAX_ITEMS = 40
MAX_TOTAL_SCORE = 6  # fund_flag(0~) + news_score(0,1,2)를 0~6 구간으로 정규화
KST = timezone(timedelta(hours=9))
try:
    # 별도 파일에 키가 있다면(예: config_secret.py), 가장 우선 사용
    import config_secret as _cfg
    OPENAI_API_KEY = getattr(_cfg, "OPENAI_API_KEY", None) or os.getenv("OPENAI_API_KEY", "")
except Exception:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

try:
    from openai import OpenAI
except Exception:
    OpenAI = None

# ===== 안전 기본값 =====
BACKUP_NEG_LEXICON = [
    "횡령","배임","분식","적자","자본잠식","상장폐지","소송","경고","리콜","징계","과징금","벌금",
    "파산","부도","유상증자","감자","유출","해킹","사고","화재","사망","구속","조사","압수수색",
    "거래정지","연체","채무불이행","영업정지","위반","부정","리스크"
]
RED_KEYWORDS = [
    "횡령","배임","분식회계","자본잠식","상장폐지","거래정지","대규모 적자","영업손실","소송","과징금",
    "제재","리콜","압수수색","유출","해킹","파산","부도","채무불이행","구속"
]

# ===== 유틸 =====
def to_kst(dt: datetime) -> datetime:
    return (dt.replace(tzinfo=timezone.utc) if dt.tzinfo is None else dt).astimezone(KST)

def parse_rfc2822_date(s: str) -> Optional[datetime]:
    try:
        from email.utils import parsedate_to_datetime
        return parsedate_to_datetime(s)
    except Exception:
        return None

def clip_days(dt: datetime, days: int = DEFAULT_DAYS) -> bool:
    return (to_kst(datetime.utcnow()) - to_kst(dt)) <= timedelta(days=days)

def _strip_basic_tags(s: str) -> str:
    return (s or "").replace("<b>", "").replace("</b>", "")

# ===== 뉴스 수집(NAVER→Google RSS 폴백) =====
def fetch_news_naver(query: str, display: int = DEFAULT_MAX_ITEMS) -> List[Dict[str, Any]]:
    if not (NAVER_ID and NAVER_SECRET):
        raise RuntimeError("NAVER 키 없음")
    headers = {
        "X-Naver-Client-Id": NAVER_ID,
        "X-Naver-Client-Secret": NAVER_SECRET,
        "User-Agent": USER_AGENT
    }
    params = {"query": query, "display": min(100, display), "start": 1, "sort": "date"}
    url = "https://openapi.naver.com/v1/search/news.json"
    r = requests.get(url, headers=headers, params=params, timeout=HTTP_TIMEOUT)
    r.raise_for_status()
    data = r.json()
    out = []
    for it in data.get("items", []):
        dt = parse_rfc2822_date(it.get("pubDate","")) or to_kst(datetime.utcnow())
        if not clip_days(dt):
            continue
        out.append({
            "title": _strip_basic_tags(it.get("title") or ""),
            "description": _strip_basic_tags(it.get("description") or ""),
            "link": it.get("link") or it.get("originallink") or "",
            "date": dt.isoformat(),
        })
        if len(out) >= DEFAULT_MAX_ITEMS:
            break
    return out

def fetch_news_google_rss(query: str, max_items: int = DEFAULT_MAX_ITEMS) -> List[Dict[str, Any]]:
    q = requests.utils.quote(query)
    url = f"https://news.google.com/rss/search?q={q}+when:{DEFAULT_DAYS}d&hl=ko&gl=KR&ceid=KR:ko"
    headers = {"User-Agent": USER_AGENT}
    r = requests.get(url, headers=headers, timeout=HTTP_TIMEOUT)
    r.raise_for_status()
    root = ET.fromstring(r.text)
    ch = root.find("channel")
    out = []
    if ch is None: return out
    for item in ch.findall("item"):
        pub = item.findtext("{http://purl.org/dc/elements/1.1/}date") or item.findtext("pubDate") or ""
        dt = parse_rfc2822_date(pub) or to_kst(datetime.utcnow())
        if not clip_days(dt):
            continue
        out.append({
            "title": item.findtext("title") or "",
            "description": item.findtext("description") or "",
            "link": item.findtext("link") or "",
            "date": dt.isoformat(),
        })
        if len(out) >= max_items:
            break
    return out

def fetch_news(query: str) -> List[Dict[str, Any]]:
    try:
        items = fetch_news_naver(query)
        if items: return items
    except Exception:
        pass
    try:
        return fetch_news_google_rss(query)
    except Exception:
        return []


# ===== 간이 감성(백업 사전 기반) =====
def analyze_sentiment_backup(texts: List[str]) -> List[float]:
    out = []
    for t in texts:
        low = (t or "").lower()
        hits = sum(1 for w in BACKUP_NEG_LEXICON if w in low)
        out.append(min(1.0, hits/3.0))  # 히트 많을수록 부정 확률↑
    return out


def smoothed_neg_ratio(neg_probs: List[float], thr: float=0.5, a: float=1.0, b: float=3.0) -> float:
    if not neg_probs: return a / (a + b)
    n = len(neg_probs); hits = sum(1 for p in neg_probs if p >= thr)
    return (hits + a) / (n + a + b)

def judge_label(neg_ratio: float) -> str:
    return "부정" if neg_ratio >= 0.60 else ("중립" if neg_ratio >= 0.30 else "긍정")


In [ ]:
from typing_extensions import Text
# ===== 펀더멘털(간이) 추정: 뉴스에서 펀더멘털성 이벤트 추출 → flag/reasons =====
def infer_fundamentals_from_news(texts: List[str]) -> Dict[str, Any]:
    """
    외부 재무 API 없이도 뉴스에서 펀더멘털 성격의 이벤트를 감지해 가점.
    대략적 기준 (최대 4점 권장):
      - 치명적: '자본잠식','부도','파산','채무불이행' → +3
      - 강함:   '감자','상장폐지','거래정지' → +2
      - 중간:   '유상증자','대규모 적자','영업손실' → +1
    컨텍스트가 불확실하면 가점을 한 단계 하향(보수화).
    """
    text = " ".join((t or "").lower() for t in texts)
    reasons: List[str] = []

    def hit(word: str) -> bool: return word in text
    def any_hit(words: List[str]) -> bool: return any(hit(w) for w in words)

    deadly = ["자본잠식","부도","파산","채무불이행"]
    strong = ["감자","상장폐지","거래정지"]
    medium = ["유상증자","대규모 적자","영업손실"]

    # 컨텍스트 동사(확정/공시/판결/개시/발생/완료/진행/임박 등)
    context_confirm = ["확정","공시","판결","개시","발생","임박","완료","진행"]
    # 완화 표현(가능성/검토/우려/관측/전망/추정/해석/소식/설 등)
    hedging_words = ["가능성","검토","논의","우려","관측","전망","추정","해석","소식","설"]

    raw = 0
    if any_hit(deadly):
        raw += 3; reasons.append("치명적 이벤트(자본잠식/부도/파산/채무불이행)")
    if any_hit(strong):
        raw += 2; reasons.append("강한 이벤트(감자/상장폐지/거래정지)")
    if any_hit(medium):
        raw += 1; reasons.append("중간 이벤트(유상증자/대규모 적자/영업손실)")

    # 컨텍스트/완화 보정
    if raw > 0 and not any_hit(context_confirm):
        raw -= 1
        reasons.append("확정이라 단언할 수는 없음")
    if raw > 0 and any_hit(hedging_words):
        raw -= 1
        reasons.append("완화될 여지 혹은 가능성 있음")

    score = max(0, min(4, raw))
    # 중복 제거
    uniq = []
    for r in reasons:
        if r not in uniq:
            uniq.append(r)
    return {"flag": score, "reasons": uniq}

# ===== 위험도 보정 (연속값 반영, 5% 고정 탈출) =====
def calibrate_risk(total_score: int,
                   max_total: int,
                   news_count: int,
                   neg_ratio: float,
                   red_hits: int = 0) -> float:
    """
    구성:
    - 규칙 점수 기반: base = 100 * (total_score / max_total)
    - 연속형 보정: news_cont = 15 * neg_ratio  (0~15%)
    - 레드키워드 보정: red_boost = min(10, 1.5 * red_hits) (0~10%)
    - 바닥치: 뉴스 있으면 5%, 없으면 2%
    → 총합에서 바닥치만 보장(강제 5% 고정 없음)
    """
    if max_total <= 0:
        raw = 15.0 * max(0.0, min(1.0, neg_ratio)) + min(10.0, 1.5 * max(0, red_hits))
    else:
        base = 100.0 * (float(total_score) / float(max_total))
        news_cont = 15.0 * max(0.0, min(1.0, neg_ratio))
        red_boost = min(10.0, 1.5 * max(0, red_hits))
        raw = base + news_cont + red_boost

    floor = 5.0 if news_count > 0 else 2.0
    return round(max(raw, floor), 1)

# ===== GPT 한줄요약 =====
GPT_SYSTEM_PROMPT = (
    "너는 한국 기업 리스크를 요약하는 애널리스트야. "
    "입력 JSON(뉴스/레드키워드/펀더멘털/라벨/위험도/주요 제목)을 보고 1~2문장으로, "
    "최대 120자 이내 한국어 존댓말로 친근하게, 중학생 수준의 쉬운 말로 요약하라. "
    "요구사항: "
    "1) 첫 문장에 레드키워드 개수와 핵심 부정 키워드를 괄호로 구체적으로 제시하라. "
    "   예: '레드키워드 1건(자본잠식/부도/파산/채무불이행)…' "
    "2) 가능하면 기사 제목에서 구체 사례 1개를 짧게 넣어라. 예: '아동 낙상 사고' "
    "3) 둘째 문장에 완화/검토/가능성 등의 표현이 기사에 보이면 '하지만 … 가능성도 있네요!'처럼 "
    "4) 출력은 오직 요약 문장만, 접두사/접미사/불릿/이모지는 쓰지 말 것."
)


def build_gpt_user_prompt(company: dict, news: dict, fundamental: dict, combined: dict, top_titles: List[str]) -> str:
    payload = {
        "company": {"name": company.get("corp_name", "")},
        "news": {
            "count": news.get("news_count", 0),
            "neg_ratio": round(news.get("neg_ratio", 0.0), 3),
            "label": news.get("label_news", ""),
            "red_hits": news.get("red_hits", 0),
            "top_titles": top_titles[:3],
        },
        "fundamental": {
            "flag": fundamental.get("flag", 0),
            "reasons": (fundamental.get("reasons") or [])[:3],
        },
        "combined": {
            "final_label": combined.get("final_label", ""),
            "risk_pct": combined.get("risk_pct", 0.0),
            "total_score": combined.get("total_score", 0),
        },
        "instruction": "부정 징후를 요인 단위로 묶어 핵심 1~2개만 나열하고 끝에 '관망' 또는 '주의'로 마무리."
    }
    return "다음 JSON을 한 줄로 요약:\n\n" + json.dumps(payload, ensure_ascii=False, indent=2)

def gpt_one_liner(company, news, fundamental, combined, top_titles) -> str:
    # 폴백(키 없음 또는 SDK 없음)
    def _fallback():
        bits = []
        if news.get("red_hits", 0) > 0:
            bits.append(f"레드키워드 {news['red_hits']}건")
        label = news.get("label_news", "")
        if label == "부정":
            bits.append("부정 기사 다수")
        if fundamental.get("flag", 0) >= 2:
            bits.append("펀더멘털 경고")
        if not bits:
            bits.append("특이 부정 신호 제한")
        tail = "관망" if (combined.get("final_label") or label) != "긍정" else "주의"
        return "·".join(bits[:2]) + f", {tail}"

    if not (OPENAI_API_KEY and OpenAI):
        return _fallback()

    client = OpenAI(api_key=OPENAI_API_KEY)

    try:
        sys_prompt = GPT_SYSTEM_PROMPT
        user_prompt = build_gpt_user_prompt(company, news, fundamental, combined, top_titles)
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": sys_prompt},
                      {"role": "user", "content": user_prompt}],
            max_tokens=80,
            temperature=0.2,
        )
        txt = (resp.choices[0].message.content or "").strip()
        return (txt.splitlines()[0] if txt else _fallback())[:120]
    except Exception as e:
        print(f"⚠️ GPT 한줄요약 오류: {e}")
        return _fallback()

#최종 라벨 함수 추가
def decide_final_label(total_score: int) -> str:
    """
    합산점수 기반 최종 라벨:
    - 0~1: 안정
    - 2~3: 양호
    - 4~ : 주의
    """
    if total_score >= 4:
        return "주의"
    if total_score >= 2:
        return "양호"
    return "안정"


# ===== 평가 파이프라인 =====
def evaluate_company(name: str) -> Dict[str, Any]:
    # 1) 뉴스 수집
    items = fetch_news(name)

    # 2) 간이 감성/레드키워드
    texts = [f"{it.get('title','')} {it.get('description','')}".strip() for it in items]
    neg_probs = analyze_sentiment_backup(texts) if texts else []
    neg_ratio = smoothed_neg_ratio(neg_probs)
    label_news = judge_label(neg_ratio)
    red_hits = 0
    for t in texts:
        low = (t or "").lower()
        red_hits += sum(1 for w in RED_KEYWORDS if w.lower() in low)

    # 3) 펀더멘털(간이) 추정 → flag/reasons
    fundamental = infer_fundamentals_from_news(texts)
    fund_flag = int(fundamental.get("flag", 0) or 0)

    # 🔼 레드키워드가 많으면 펀더멘털 가점(최대 1점) — 실무적 프록시
    if red_hits >= 3:
        fund_flag = min(4, fund_flag + 1)
        reasons = fundamental.get("reasons") or []
        reasons.append("부정 키워드 다수(3건 이상)")
        fundamental["reasons"] = reasons
        fundamental["flag"] = fund_flag

    # 4) 종합 점수 및 위험도 보정 ← red_hits를 같이 반영
    news_score = {"긍정": 0, "중립": 1, "부정": 2}[label_news]
    total_score = fund_flag + news_score
    risk_pct = calibrate_risk(total_score, MAX_TOTAL_SCORE, len(items), neg_ratio, red_hits=red_hits)

    # 🔽 risk_pct 계산 후에 최종 라벨 결정
    final_label = decide_final_label(total_score)

    # 위험도 가드레일
    if risk_pct >= 60.0:
      final_label = "부정"
    elif risk_pct >= 35.0 and final_label == "긍정":
      final_label = "중립"

# 5) 컴포넌트 dict들
    company = {"corp_name": name, "corp_code": "", "stock_code": ""}
    news = {
        "news_count": len(items),
        "neg_ratio": neg_ratio,
        "label_news": label_news,
        "red_hits": red_hits
    }

    # 6) GPT 한 줄 요약
    top_titles = [it.get("title", "") for it in items][:3]
    combined = {
        "final_label": final_label,    # ← risk_pct 가드레일 반영된 라벨
        "total_score": total_score,    # ← 펀더멘털 flag + 뉴스 점수
        "risk_pct": risk_pct,          # ← calibrate_risk 결과
        "one_liner": gpt_one_liner(company, news, fundamental, {
            "final_label": final_label,
            "total_score": total_score,
            "risk_pct": risk_pct,
        }, top_titles)                 # ← 프롬프트 기반 GPT 한줄요약
    }


    # 7) 결과 반환
    return {
        "company": company,
        "news": news,
        "fundamental": fundamental,
        "combined": combined,
        "items": items
    }

# ===== CLI(노트북/서버 공통) =====
def main():
    print("🔎 검색할 기업명 일부를 입력하세요:", flush=True)
    try:
        keyword = input("> ").strip()
    except EOFError:
        print("❌ 표준입력 없음"); return
    if not keyword:
        print("❌ 기업명 키워드를 입력하세요."); return

    result = evaluate_company(keyword)
    corp = result["company"]; news = result["news"]; fundamental = result["fundamental"]; combined = result["combined"]

    # print(f"\n📰 수집된 뉴스: {news['news_count']}건 (최근 {DEFAULT_DAYS}일)")
    # print("\n===== 결과 =====")
    # print(f"기업: {corp['corp_name']} (비상장, corp_code {corp['corp_code']})")
    # # print(f"[뉴스] 부정비율(스무딩): {news['neg_ratio']*100:.1f}% | 레드키워드: {news['red_hits']}")
    # print(f"[리스크 조언] flag: {fundamental.get('flag',0)} | 사유: {', '.join(fundamental.get('reasons',[])) or '없음'}")
    # # print(f"[종합판정] 라벨: {combined['final_label']} | 총점: {combined['total_score']}/{MAX_TOTAL_SCORE} | 위험도(정규화): {combined['risk_pct']:.1f}%")
    # # print(f"한줄요약: {combined.get('one_liner','')}")

    # 부정 확률 상위 기사 (최대 50개)
    top_neg = sorted(
        zip(analyze_sentiment_backup([f"{i.get('title','')} {i.get('description','')}".strip() for i in result["items"]]),
            result["items"]),
        key=lambda x: x[0], reverse=True
    )[:50]

    # if top_neg:
    #     print("\n⚠️ 부정 확률 높은 기사 Top 50")
    #     for p, it in top_neg:
    #         dt = it.get("date","")[:19].replace("T"," ")
    #         print(f"- {p*100:5.1f}% | {dt} | {it.get('title','').strip()}")

    # 색상·리스크 관련 코멘트
    mapping = {"안정": "✅ 안정 - 초록색", "양호": "🟨 양호 - 노란색", "주의": "🟥 주의 - 빨간색"}
    print("\n" + mapping.get(combined["final_label"], "ℹ️ 알 수 없음 - 회색"),
          f"(잠재적 위험도 {combined['risk_pct']:.1f}%)")
    print(f"[리스크 조언] {', '.join(fundamental.get('reasons',[])) or '없음'}")

if __name__ == "__main__":
    main()

🔎 검색할 기업명 일부를 입력하세요:
> 한화

🟨 양호 - 노란색 (잠재적 위험도 36.7%)
[리스크 조언] 치명적 이벤트(자본잠식/부도/파산/채무불이행), 완화될 여지 혹은 가능성 있음
